# HARP
## Have Another Retirement Planner

This package is a laboratory for exploring the sensitivity of retirement financial decisions. It is not a planning tool, but more an environment to explore *what if* scenarios.

Diclaimer: *You make your own decisions and I am not a financial planner. Use at your own risks.*

### Just some Python module bookkeeping

In [ ]:
%matplotlib inline
# import harp as rp

import importlib
rp = importlib.import_module('harp')
importlib.reload(rp)

rp.setVerbose(True)

## Initialize our plan
Plan must start with providing the year of birth of each of the spouse(s) and their expected lifespan(s).

There are plenty of longivity predictors on the web. Pick your favorite:

https://www.livingto100.com/calculator

https://www.sunlife.ca/en/tools-and-resources/tools-and-calculators/life-expectancy-calculator/

or just Google life expectancy calculator.

There are two values needed for couples. Single individuals just enter one value in each list [ ]. For couples, keep the same order in the pair of values when entering the data. Here Jack was born in 1961 and Jill in 1964. Jack hopes to live to 89 years old, while Jill thinks she might reach age 92.

In [ ]:
plan = rp.Plan(YOB=[1961, 1964], expectancy=[89, 92])

## Specify account balances and spousal beneficiaries
For each spouse, accounts have three buckets comprising of the total value of:
- Individual taxable investment accounts, including bank accounts, and CDs
- Tax-deferred accounts, including all IRAs, 401k, 403b, etc.
- Tax-free accounts, including Roth IRAs and Roth 401k

For spouses, there are two values for each type of account, one for each spouse following the same order as before. For single individuals, only one value needs to be entered between each [ ].

The beneficiary values specify the fraction of total of assets left to the other spouse at death. For example, a spouse leaving 3/4 of her fortune to her three children and the other part to her partner would have a beneficiary value of 0.25.

Jack and Jill leave everything to each other as account beneficiaries ([1, 1]). Jack has \\$90k in his taxable account, \\$500k in his 401k, and \\$80 in his Roth 401k and Roth IRAs. Jill has \\$40k in her bank account, \\$150k in a 403b, and \\$25k in Roth IRAs in which she contributed over the years.

In [ ]:
plan.setAssetBalances(taxable=[90000, 40000], 
                      taxDeferred=[500000, 150000],
                      taxFree=[80000, 25000],
                      beneficiary=[1., 1.])

## What are current and future asset allocations?
Each account can invest in 4 major asset classes:
- Equity funds tracking the S&P500 index
- Bond assets tracking the Corporate bonds (Baa) index
- Fixed-income securities represented by 10-year Treasury bonds
- Inflation-indexed securities tracking urban Consumer Price Index (common assets)

The total of percentages in each class of asset for each account must add to 100%.

You are asked to provide an asset distribution for today, and one for the end of your life.
Values in between will be interpolated using a linear operator (for now). This can be useful
if you want to shift asset allocation as you age.

In the example below, allocation starts in full stock equities in the tax-free account,
gradually transitioning to a traditional 60/40 portfolio towards the end of life. 

In [ ]:
plan.setInitialAR(taxableAR=[[0, 25, 50, 25], [0, 25, 50, 25]],
                  taxDeferredAR=[[60, 40, 0, 0], [60, 40, 0, 0]],
                  taxFreeAR=[[100, 0, 0, 0], [100, 0, 0, 0]])

plan.setFinalAR(taxableAR=[[0, 25, 50, 25], [0, 25, 50, 0]],
                taxDeferredAR=[[60, 40, 0, 0], [60, 40, 0, 0]],
                taxFreeAR=[[60, 40, 0, 0], [60, 40, 0, 0]])

plan.interpolateAR('linear')

## Specify rates of return and inflation rate
Rates of return can be specified for each class of assets, and inflation.
Valid entries are 'historical', 'stochastic', or a list of average values.

For historical and stochastic data, data from 1928 to today is available for experimenting.
Ranges chosen smaller that the life horizon will have rate values repeated in cycle. For example,
choosing historical data from 1994 to 1996 will repeat these three values over the timespan of the plan.

Choosing a historical range of 1970 to 2022, will use the rates of 1970 for this year and 1971 for next, etc. Worst-case scenario is a retirement starting in 1966. It can be simulated as follows:

    plan.setRates('historical', 1966, 2022)
    
Stochastic data on the other hand computes the multivariate distributions for the 4 rates in a range selected. For example,

    plan.setRates('stochastic', 1945, 2022)
    
will analyze the data from 1945 to 2022 and compute means and covariance to generate statistically similar results. The stochastic rates generated for the timespan can be plotted as we will see below.
    
One can also use fixed rates by providing a list of 4 entries as follows:

    myrates = [9.6, 4.0, 3.0, 3.8]
    plan.setRates(myrates)
    
This example would use a fixed average rates of 9.6%, 4% and 3% average return on S&P500, corporate bonds, Treasury bonds, and common assets, respectively, with an average inflation rate of 3.8%. Common assets are investments tracking inflation only.

Note that the S&P500 rates provided always include dividends.

In [ ]:
plan.setRates('historical', 1969, 2022)
# plan.setRates([8, 4, 4, 3])

## What is your desired net income at retirement?
For determining this value, retirement planners will strongly suggest that you've must have done a cashflow analysis on your yearly spending. After this exercise, you should have a good idea of how much you'll need in retirement.

The desired income here is the net income (i.e., after paying taxes) that you would like to have starting at your retirement age. This income is adjusted for inflation and can follow an additional adjustment called a *smile* profile. A *smile* profile accounts for the fact that your spending capacity will modulate during retirement as you go from the so-called gogo years to the no-go years. A *flat* profile, on the other hand, will keep the same value, which will only be inflation-adjusted.

The target and actual net income values achieved by the plan can be plotted as will see below.

In [ ]:
# Enter desired value for net income, and spending profile.
plan.setDesiredIncome(75000, 'smile')

## You must also have some fixed income?
Pension and social security are fixed income. Model here assumes that pension is not inflation adjusted while social security benefits are (but code can easily be modified to account for inflation-adjusted pensions). Entries are the predicted amount for each spouse and the age of the commencement of benefits.

If you have no pension, just use zeros (0) as entries, as in

    plan.setPension([0, 0], [65, 65])
    
For social security, you must provide the predicted amount(s) and the age(s) at which you start receiving benefits. There are plenty of social security benefit estimators on the web, including the info you can get directly from your own account at the Social Security Administration (ssa.gov).

Here Jill has an unindexed pension of \\$10k per year. Both Jack and Jill decided to take their social security benefits at age 70. The amount provided are estimation of the amounts they would receive at age 70. 

In [ ]:
plan.setPension([0, 10000], [65, 65])

plan.setSocialSecurity([30000, 28000], [70, 70])

## Provide work income, contributions, Roth conversions, and other items by year
In order to get a plan, you must provide an Excel workbook with one spreadsheet (tab) per spouse with the following information:

|year|anticipated income|ctrb taxable | ctrb 401k | ctrb Roth 401k | ctrb IRA | ctrb Roth IRA | Roth X | big ticket items|
|--|--|--|--|--|--|--|--|--|
|2023 | | | | | | | | |
|2024 | | | | | | | | |
| ... | | | | | | | | |
|20XX | | | | | | | | |

Here, 20XX is the first year after both spouses have passed. For the columns, *anticipated income* is the annual amount (gross) that you anticipate to receive from employement or other sources (not including dividends from your taxable investment account). Note that column names are case sensitive and all of these entries must be in lower case.

Contributions to your savings accounts are marked as *ctrb*. Contributions to your 401k must also include your employer's contributions. Being in excel, you can use the calculator to enter percentage of your anticipated income if that is easier for you. 
Files format from the LibreOffice software can also be read.

Roth conversion are specified in the column marked *Roth X*. Finally, *big ticket items* are the sale or purchase of a house, or any other major expense or money that you would give or receive (e.g., inheritance, or large gifts to or from you). Therefore, the sign (+/-) of entries in this column is important. All other column entries should be positive.

The TAB name for each spreadsheet represents the name of the spouse for reporting transactions affecting your plan. So for Jack and Jill, this worksheet file would have two tabs, the first one named *Jack* and the second one named *Jill*, following the same order of data provided before.

An Jack and Jill provided their information in the file *jack+jill.xlsx*.

In [ ]:
plan.readContributions('jack+jill.xlsx')

## How do you want to withdraw from you and your spouse's accounts?
By specifying an *auto* spousal split, the withdrawals from the retirement accounts will be made proportial to their respective balances. It can also be specified as a fractional value (e.g., 0.65) in which case 0.65 will be taken from the first account and the other 35% taken from the other (second entry) spouse.

All withdrawals use a smart banking approach which favors depleting taxable accounts before tax-deferred, before tax-free. With *auto*, additional checks and bounds are used to coordinate the spousal accounts.

In [ ]:
plan.setSpousalSplit('auto')

## Run calculations
We're ready!

This call runs the calculations for the time horizon set by the life expectancy of individual(s).

In [ ]:
plan.run();

# Analysis

### Show net income compared to target over the years

In [ ]:
plan.plotNetIncome();

### Show sources of income over the years

In [ ]:
plan.plotSources();

### Show account balances at the beginning of each year

In [ ]:
plan.plotAccounts();

### Show annual taxes paid over the years

In [ ]:
plan.plotTaxes();

### Show taxable annual income and anticipated tax brackets

In [ ]:
plan.plotTaxableIncome();

### Show annual rates used for calculations

In [ ]:
plan.plotRates();

## Do we save this plan?
The plan contains a lot of information on the distribution amounts, including the required minimum distribution that had to be performed under the given assuptions. This info can be saved in an excel workbook with one spreadsheet (tab) for Jack and one for Jill.

In [ ]:
# plan.savePlanXL('Jack and Jill')